### Import libraries and define subject path

In [1]:
# This is our preprocessing script.
# We select one recording, clean stuff and then perform the ICA
# During this process, we save all bad components segments and channels

import sys
import os.path as op
module_path = op.abspath(op.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
import mne
from autoreject import AutoReject, get_rejection_threshold

from data_analysis.functions_preprocessing import \
    (split_raws, mark_bads, save_bads, run_ica, save_ica, save_autoreject)
from data_analysis.functions_behavioral import \
    (create_event_df, remove_ghost_triggers, calculate_alpha,
     join_event_dfs, remove_outliers, events_from_event_df)

subject_dir = '/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/'
behav_dir = "/net/store/nbp/projects/hyperscanning/study_project/NBP_Hyperscanning/data_analysis/Behavioural_Analysis/BehaviouralData"


#%matplotlib notebook
#%matplotlib tk
#%matplotlib qt
#%matplotlib ipympl
%matplotlib widget

import matplotlib
#matplotlib.use('Qt5Agg')
matplotlib.get_backend()


Bad key "text.kerning_factor" on line 4 in
/net/store/nbp/projects/hyperscanning/study_project/programming_tools/miniconda3/envs/hyperscanning/lib/python3.8/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


'module://ipympl.backend_nbagg'

## Define the subject you want to clean

In [2]:
subj_pair = input("Please Type in, which subject pair you want to clean.\n"
                  "For the pilot study, possible choices are:\n"
                  "[202, 203, 204, 205, 206, 207, 208, 209, 211, 212]\n")

participant = input("\nPlease Type in, which subject pair you want to clean.\n"
                    "Type: 0 for the first participant and: 1 for the second.\n")

Please Type in, which subject pair you want to clean.
For the pilot study, possible choices are:
[202, 203, 204, 205, 206, 207, 208, 209, 211, 212]
202

Please Type in, which subject pair you want to clean.
Type: 0 for the first participant and: 1 for the second.
0


## Load and prepare the EEG recording

In [3]:
# define the subjects id and its path
subj_id = "sub-{0}_p-{1}".format(subj_pair, participant)
subs_path = subject_dir + "sub-{0}/eeg/sub-{0}_task-hyper_eeg.fif".format(subj_pair)
behav_path = op.join(behav_dir, "{0}.csv".format(subj_pair))

## overwrite it for the test
# TODO: This line should be removed for the actual cleaning
subj_id = "test_2"

# load the data
combined_raw = mne.io.read_raw_fif(subs_path, preload=True)

# split the subjects and delete the raw file
raw = split_raws(combined_raw)[int(participant)]
del combined_raw

# set reference
raw.set_eeg_reference(["Cz"])

# set the EEG Montage. We use 64 chans from the standard 10-05 system.
montage = mne.channels.make_standard_montage("standard_1005")
raw.set_montage(montage)

# filter ?
raw.filter(l_freq=0.1, h_freq=120)
raw.notch_filter(freqs=[16.666666667, 50]) # bandstop the train and power grid

Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif...


<ipython-input-3-2691fcbf0139>:11: RuntimeWarning: This filename (/net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz or _meg.fif
  combined_raw = mne.io.read_raw_fif(subs_path, preload=True)


Isotrak not found
    Range : 0 ... 3725311 =      0.000 ...  3637.999 secs
Ready.
Opening raw data file /net/store/nbp/projects/hyperscanning/hyperscanning-2.0/mne_data/sourcedata/sub-202/eeg/sub-202_task-hyper_eeg-1.fif...
Isotrak not found
    Range : 3725312 ... 5675445 =   3638.000 ...  5542.427 secs
Ready.
Reading 0 ... 5675445  =      0.000 ...  5542.427 secs...
EEG channel type selected for re-referencing
Applying a custom EEG reference.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 1.2e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 120.00 Hz
- Upper transition bandwidth: 30.00 Hz (-6 dB cutoff frequency: 135.00 Hz)
- Filter length: 337

<Raw | sub-202_task-hyper_eeg.fif, 80 x 5675446 (5542.4 s), ~3.38 GB, data loaded>

## Define events and epoch the data

In [4]:
# do the behavioral analysis and get the epochs
behavioral_df = calculate_alpha(pd.read_csv(behav_path))

event_df = create_event_df(raw)
event_df = remove_ghost_triggers(event_df)
event_df = join_event_dfs(event_df, behavioral_df)
event_df = remove_outliers(event_df, exclude_stddev=2)
events = events_from_event_df(event_df)

# define the parameters for epoching
# TODO: Define events more elaborate!
event_id = 11
tmin = 0
tmax = 3

epochs = mne.Epochs(raw, events, event_id, tmin, tmax,
                    picks=["eeg"], baseline=(0, 0), preload=True)

6648 events found
Event IDs: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33 34 36 37 38 39 40 41 42 43 44 45 46 48 49]
288 matching events found
Applying baseline correction (mode: mean)
Not setting metadata
0 projection items activated
Loading data for 288 events and 3073 original time points ...
0 bad epochs dropped


## Run autoreject on the data

In [5]:
# define an autoreject object
ar = AutoReject(consensus=[0.1, 0.2, 0.3, 0.4, 0.5], thresh_method='random_search', verbose="tqdm_notebook")

# fit the epochs
ar.fit(epochs)

# plot it
ar.get_reject_log(epochs).plot()

# get the rejection threshold for ICA
reject = get_rejection_threshold(epochs)

Running autoreject on ch_type=eeg








Estimated consensus=0.30 and n_interpolate=4


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Estimating rejection dictionary for eeg


## Save the autoreject object

In [6]:
save_autoreject(ar, subj_id)

## Run (or load) the ICA and plot all components

In [7]:
# filter again for ICA
epochs.filter(l_freq=2, h_freq=None)

# run the ICA and save the marked components
ica = run_ica(epochs, subj_id, n_components=25, method="fastica", reject=reject)

Setting up high-pass filter at 2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Filter length: 1691 samples (1.651 sec)

Path to ICA file already exists.
Delete the respective ICA file (/net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/bads/bad_components/test_2-ica.fif)
if you want to fit a new ICA.
 Loading existing ICA
Reading /net/store/nbp/projects/hyperscanning/study_project/dirk/NBP_Hyperscanning/data_analysis/bads/bad_components/test_2-ica.fif ...
Now restoring ICA solution ...
Ready.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Choose specific component properties to inspect

In [8]:
inp = input("Please type in which components you want to further "
            "inspect.\nE.G. 3, 4,15 for components 3, 4, and 15.\n")

inp = [int(n) for n in inp.split(",") if n != ""]

if len(inp) > 0:
    ica.plot_properties(epochs, picks=inp, reject=None)


Please type in which components you want to further inspect.
E.G. 3, 4,15 for components 3, 4, and 15.
3


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

    Using multitaper spectrum estimation with 7 DPSS windows
288 matching events found
No baseline correction applied
Not setting metadata
0 projection items activated
0 bad epochs dropped


## Exclude specific ICA components

In [9]:
print("Excluded ICA components: ", ica.exclude)

inp = input("\nPlease type in which components you want to exclude.\n"
            "E.G. 2, 3,14 for components 2, 3, and 14.\n")

bad_comps = [int(comp) for comp in inp.split(",") if comp != ""]
bad_comps = [comp for comp in set(bad_comps) if comp not in ica.exclude]

ica.exclude.extend(bad_comps)
print("\nExcluded ICA components: ", ica.exclude)

Excluded ICA components:  [1, 2, 3, 4, 5]

Please type in which components you want to exclude.
E.G. 2, 3,14 for components 2, 3, and 14.
2

Excluded ICA components:  [1, 2, 3, 4, 5]


## Save the ICA and its excluded components

In [10]:
inp = input("Do you really want to save the components?\n"
            "Enter 'save' or 's' to save the data. Else, "
            "changes will be discarded.\n")

if inp[0] == "s":
    save_ica(ica, subj_id)

Do you really want to save the components?
Enter 'save' or 's' to save the data. Else, changes will be discarded.
l


### give everyone access to the new marked files you've created

In [11]:
!cd /net/store/nbp/projects/hyperscanning/study_project
!chown -hR $USER:nbp *; chmod -R 770 *



### Everything done. Thanks for cleaning :)